# Filtering Ukrainian artists

In [4]:
import pandas as pd
import os

songs_df = pd.read_csv(os.path.join("data", "songs_v1.csv"))
print("Length before filtering: ", len(songs_df))

Length before filtering:  10854


1. Drop duplicates:

In [69]:
songs_df = songs_df.drop_duplicates(subset=["title", "artist"])
print("Length after dropping duplicates: ", len(songs_df))

Length after dropping duplicates:  10804


2. Filter non ukr artists and dublicates of the same artist:

In [70]:
ARTISTS_BLACKLIST = [
    "Poshlaya Molly", "Poshlaja Molli", "Сметана band", "Corn Wave", "Потап И Настя Каменских", 
    "Светлана Лобода", "Valentin Strykalo", "Верка Сердючка", "Lюk", "kavabanga Depo kolibri", 
    "Quest Pistols Show", "Какая Разница", "Green Grey", "jockii druce", "Миша Марвин", 
    "Оля Полякова", "Михаил Бублик", "Алёна Винницкая", "Наталья Могилевская",
    "Таисия Повалий", "Сергій Жадан Та Собаки В Космосі", "Poshlaya Molly",
    "Valentin Strykalo", "INGRET", "Svitlana Nianio", "Vasily Richter",
    "MARUV & Boosin", "Сусіди Стерплять", "The Feels", "Grandma's smuzi", 
    "Табула Раса", "Олег Кензов", "Potap & Nastya", "Виталий Чирва", "TELLY GRAVE",
    "Анастасия Приходько", "glichery", "AShamaluevMusic", "daKooka", "4Wheel", "KLIM", "CLONNEX", 
    "bris", "Pencil Legs", "Mykola Dmytrovych Leontovych", "Odyn v kanoe", 
    "Святослав Вакарчук", "DJ Jedy", "Марія Чайковська", "The Ukrainians", "Los Colorados", 
    "Тіна Кароль", "Ірина Білик", "national radio"
    ]

songs_df = songs_df[~songs_df["artist"].isin(ARTISTS_BLACKLIST)]
print("Length after removing blacklisted artists: ", len(songs_df))

Length after removing blacklisted artists:  5872


3. Remove songs that have brackets in the title (probably remixes of the original songs) or are too long:

In [71]:
songs_df = songs_df[~songs_df["title"].str.contains("\(") | ~songs_df["title"].str.contains("\)")]
songs_df = songs_df[~songs_df["title"].str.contains("\[") | ~songs_df["title"].str.contains("\]")]
songs_df = songs_df[~songs_df["title"].str.contains("\|")]
songs_df = songs_df[~songs_df["title"].str.contains("\.")]
songs_df = songs_df[songs_df["title"].str.len() < 30]

print("Length after filtering based on song title: ", len(songs_df))

Length after filtering based on song title:  4417


4. Filter non ukr artists based on the songs' tags:

Note: don't include russian here as a considerable amount of ukr songs have a missleading russian tag

In [72]:
TAGS_BLACKLIST = ["finnish", "japanese"]
songs_df = songs_df[~songs_df["title_tags"].str.contains("|".join(TAGS_BLACKLIST))]
print("Length after filtering based on title tags: ", len(songs_df))

Length after filtering based on title tags:  4341


Save the filtered songs to a new *.csv file:

In [ ]:
songs_df.to_csv(os.path.join("data", "songs_filtered_v1.csv"), index=False)

## Data summary after filtering

In [93]:
import pandas as pd
import os

filtered_songs_df = pd.read_csv(os.path.join("data", "songs_filtered_v1.csv"))

In [97]:
print("Number of ukr artists: ", len(filtered_songs_df["artist"].unique()))

Number of ukr artists:  62


In [98]:
songs_per_artist = filtered_songs_df["artist"].value_counts()

print(f"Total number of songs: {len(filtered_songs_df)}\n")
print(f"Number of songs per artist:\n{songs_per_artist.describe()}")

Total number of songs: 4341

Number of songs per artist:
count     62.000000
mean      70.016129
std       21.063909
min       18.000000
25%       53.000000
50%       75.500000
75%       87.750000
max      100.000000
Name: artist, dtype: float64


Save the current blacklist version to a *.txt file":

In [ ]:
with open(os.path.join("configs", "artists_blacklist_v1.txt"), "w", encoding="utf-8") as f:
    f.write("\n".join(ARTISTS_BLACKLIST))

## Concat with YT fetched results and update summary

In [ ]:
import pandas as pd
import os

filtered_songs_df = pd.read_csv(os.path.join("data", "songs_filtered_v1.csv"))
len(filtered_songs_df)

4341

In [ ]:
audios_5_df = pd.read_csv(os.path.join("metadata", "yt_songs_5_2_pages_filtered.csv"))
audios_10_df = pd.read_csv(os.path.join("metadata", "yt_songs_10_2_pages_filtered.csv"))
audios_more_df = pd.read_csv(os.path.join("metadata", "yt_songs_more_sooongs_filtered.csv"))

audios_df = pd.concat([audios_5_df, audios_10_df, audios_more_df], ignore_index=True)

print("Total: ", len(audios_df))
print("No yt_url: ", len(audios_df[audios_df["yt_url"].isnull()]))

Total:  4068
No yt_url:  602


In [89]:
merged_df = pd.merge(filtered_songs_df, audios_df, on=["title", "artist"], how="left")

print("Columns: ", merged_df.columns.values)
print("Total: ", len(merged_df))
print("No yt_url: ", len(merged_df[merged_df["yt_url"].isnull()]))

Columns:  ['title' 'artist' 'title_listeners_x' 'title_scrobbles_x' 'title_tags_x'
 'title_duration_x' 'title_listeners_y' 'title_scrobbles_y' 'title_tags_y'
 'title_duration_y' 'yt_title' 'yt_url' 'yt_duration' 'yt_views'
 'audio_path']
Total:  4341
No yt_url:  886


In [ ]:
merged_df[["title", "artist", "audio_path", "yt_title", "yt_url", "yt_duration", "yt_views"]]\
    .to_csv(os.path.join("data", "yt_songs_filtered_v1.csv"), index=False)

Updated summary after fetching the missing songs:

In [3]:
import pandas as pd
import os

filtered_songs_df = pd.read_csv(os.path.join("data", "yt_songs_filtered_v1.csv"))

print("Columns: ", filtered_songs_df.columns.values)
print("Total: ", len(filtered_songs_df))
print("No yt_url: ", len(filtered_songs_df[filtered_songs_df["audio_path"].isnull()]))

Columns:  ['title' 'artist' 'audio_path' 'yt_title' 'yt_url' 'yt_duration'
 'yt_views']
Total:  4341
No yt_url:  60


# Exploring non unique audio_path

In [5]:
import pandas as pd
import os

yt_df = pd.read_csv(os.path.join("data", "yt_songs_filtered_v1.csv"))

len(yt_df["audio_path"].unique())

3227

In [21]:
non_unique_paths = yt_df["audio_path"].value_counts()[yt_df["audio_path"].value_counts() > 1].index.tolist()
non_unique_paths

['audio\\Нумер 482 - Добрий ранок Україно - (Офіційний кліп- 2015).mp3',
 'audio\\Ігор Кайдаш - цього вартує кохання (official mood video).mp3',
 'audio\\Димна Сумiш - Зламанi.mp3',
 'audio\\Серцевий Напад - Субкультура.mp3',
 'audio\\Нумер 482 - Важлива | Official Video.mp3',
 'audio\\Крихітка Цахес - На Першому Місці.mp3',
 "audio\\Її душі зів'ялі квіти.mp3",
 'audio\\◾Плач Єремії ◾ Вона ◾.mp3',
 'audio\\Лента За Лентою.mp3',
 'audio\\Лінія Маннергейма – Де твоя лінія?.mp3',
 'audio\\Холодне Сонце - Тінь Кохання (remastered HQ).mp3',
 'audio\\Один в каное - Подобається, як ти ідеш.mp3',
 'audio\\Воплі Відоплясова - ВЕСНА.mp3',
 'audio\\Воплі Відоплясова - Були на селі.mp3',
 'audio\\Катя Chilly - Я - молодая [HD 720p].mp3',
 'audio\\Плач Єремії - Ти втретє цього літа зацвітеш.mp3',
 'audio\\Sashakhuy.mp3',
 'audio\\Ало-але.mp3',
 'audio\\Zalyshai.mp3',
 'audio\\Воплі Відоплясова - Сонячні дні [Official Video].mp3',
 'audio\\The Unsleeping — PTAHA FRED (Official Video).mp3',
 'audio\\

It seems like we have two cases:
1. It is literaly the same song but with slightly different title name in the LastFM platform

In [8]:
yt_df[yt_df["audio_path"] == 'audio\\Ало-але.mp3']

,title,artist,audio_path,yt_title,yt_url,yt_duration,yt_views
752,Ало-але,Крихітка Цахес,audio\Ало-але.mp3,Ало-але,https://youtube.com/watch?v=xH6cxIXtGWI,222.0,172200.0
805,Ало але,Крихітка Цахес,audio\Ало-але.mp3,Ало-але,https://youtube.com/watch?v=xH6cxIXtGWI,222.0,172201.0
812,"Алло, але",Крихітка Цахес,audio\Ало-але.mp3,Ало-але,https://youtube.com/watch?v=xH6cxIXtGWI,222.0,172201.0
813,"Твої ало, мої але",Крихітка Цахес,audio\Ало-але.mp3,Ало-але,https://youtube.com/watch?v=xH6cxIXtGWI,222.0,172201.0
818,"Ало, але",Крихітка Цахес,audio\Ало-але.mp3,Ало-але,https://youtube.com/watch?v=xH6cxIXtGWI,222.0,172201.0


2. Those are different songs, but the script failed to search the relevant YT audio (in some cases it doesn't even exist on YT)

In [73]:
yt_df[yt_df["audio_path"] == 'audio\\Плач Єремії - Ти втретє цього літа зацвітеш.mp3']

,title,artist,audio_path,yt_title,yt_url,yt_duration,yt_views,title_norm
1659,Ти втретє цього літа зацвітеш,Плач Єремії,audio\Плач Єремії - Ти втретє цього літа зацві...,Плач Єремії - Ти втретє цього літа зацвітеш,https://youtube.com/watch?v=ergjjMQywHU,217.0,2045107.0,Ти втретє цього літа зацвітеш
1685,Літо,Плач Єремії,audio\Плач Єремії - Ти втретє цього літа зацві...,Плач Єремії - Ти втретє цього літа зацвітеш,https://youtube.com/watch?v=ergjjMQywHU,217.0,2045107.0,Літо
1704,Ти втретє цього лiта зацвітеш,Плач Єремії,audio\Плач Єремії - Ти втретє цього літа зацві...,Плач Єремії - Ти втретє цього літа зацвітеш,https://youtube.com/watch?v=ergjjMQywHU,217.0,2045107.0,Ти втретє цього літа зацвітеш
1705,Ти втретє цього лiта зацвiтеш,Плач Єремії,audio\Плач Єремії - Ти втретє цього літа зацві...,Плач Єремії - Ти втретє цього літа зацвітеш,https://youtube.com/watch?v=ergjjMQywHU,217.0,2045107.0,Ти втретє цього літа зацвітеш
1725,"Ті, що забули",Плач Єремії,audio\Плач Єремії - Ти втретє цього літа зацві...,Плач Єремії - Ти втретє цього літа зацвітеш,https://youtube.com/watch?v=ergjjMQywHU,217.0,2045108.0,"Ті, що забули"
3260,Ти втретє цього літа зацвітеш,Тарас Чубай,audio\Плач Єремії - Ти втретє цього літа зацві...,Плач Єремії - Ти втретє цього літа зацвітеш,https://youtube.com/watch?v=ergjjMQywHU,217.0,2050516.0,Ти втретє цього літа зацвітеш


We need to perform two further postprocessing steps:
1. Normalize artist and titles and filter duplicates
2. Prepare a column which will indicate the need for re-downloading the audio (or setting None if it doesn't exist on YT)

## Artist normalization

In [14]:
print("Number of unique artists:", len(yt_df['artist'].unique()))
yt_df['artist'].unique()

Number of unique artists: 60


array(['Океан Ельзи', 'Go_A', 'Бумбокс', 'The Hardkiss', 'alyona alyona',
       'Один В Каное', 'Друга Ріка', 'Vivienne Mort', 'Воплі Відоплясова',
       'Мертвий Півень', 'Крихітка Цахес', 'Жадан і собаки',
       'Брати Гадюкіни', 'The Unsleeping', 'Artem Pivovarov',
       'СТРУКТУРА ЩАСТЯ', 'Кому Вниз', 'Димна Суміш', 'Klavdia Petrivna',
       'Tember Blanche', 'Khrystyna Soloviy', 'dity inzheneriv',
       'PROBASS ∆ HARDI', 'Плач Єремії', 'Холодне Сонце', 'Sad Novelist',
       'Скрябін', 'Перкалаба', 'Zlata Ognevich', 'Mad Heads XL',
       'Kalush Orchestra', 'Піккардійська Терція', 'BLOOMS CORDA',
       'Ницо Потворно', 'Фактично Самі', 'renie cares', 'Обійми Дощу',
       'Лінія Маннергейма', 'Арсен Мірзоян', 'Маша Кондратенко',
       'Олександр Пономарьов', 'Гурт [О]', 'Скажи щось погане',
       'Без Обмежень', 'Ігор Кайдаш', 'Тарас Чубай', 'Серцевий Напад',
       'Орест Лютий', 'Нумер 482', 'Марія Бурмака', 'Катя Chilly',
       'To Eternity', 'Dazzle Dreams', 'KALUS

Duplicates in the artists list:
- Zhadan i Sobaky - Жадан і собаки
- Воплi Вiдоплясова - Воплі Відоплясова
- Kalush Orchestra - Kalush (UPD: separate artists)

In [6]:
def normalize_artist(df):
    df['artist'] = df['artist'].replace('Воплi Вiдоплясова', 'Воплі Відоплясова')
    df['artist'] = df['artist'].replace('Zhadan i Sobaky', 'Жадан і собаки')
    return df

In [ ]:
yt_df = normalize_artist(yt_df)
print("Number of unique artists after normalization:", len(yt_df['artist'].unique()))
yt_df['artist'].unique()

Number of unique artists after normalization: 60


array(['Океан Ельзи', 'Go_A', 'Бумбокс', 'The Hardkiss', 'alyona alyona',
       'Один В Каное', 'Друга Ріка', 'Vivienne Mort', 'Воплі Відоплясова',
       'Мертвий Півень', 'Крихітка Цахес', 'Жадан і собаки',
       'Брати Гадюкіни', 'The Unsleeping', 'Artem Pivovarov',
       'СТРУКТУРА ЩАСТЯ', 'Кому Вниз', 'Димна Суміш', 'Klavdia Petrivna',
       'Tember Blanche', 'Khrystyna Soloviy', 'dity inzheneriv',
       'PROBASS ∆ HARDI', 'Плач Єремії', 'Холодне Сонце', 'Sad Novelist',
       'Скрябін', 'Перкалаба', 'Zlata Ognevich', 'Mad Heads XL',
       'Kalush Orchestra', 'Піккардійська Терція', 'BLOOMS CORDA',
       'Ницо Потворно', 'Фактично Самі', 'renie cares', 'Обійми Дощу',
       'Лінія Маннергейма', 'Арсен Мірзоян', 'Маша Кондратенко',
       'Олександр Пономарьов', 'Гурт [О]', 'Скажи щось погане',
       'Без Обмежень', 'Ігор Кайдаш', 'Тарас Чубай', 'Серцевий Напад',
       'Орест Лютий', 'Нумер 482', 'Марія Бурмака', 'Катя Chilly',
       'To Eternity', 'Dazzle Dreams', 'KALUS

UPD: can be performed without hardcoding via ```cyrtranslit``` library:

In [17]:
import cyrtranslit

print(cyrtranslit.to_cyrillic("Zhadan i Sobaky", "ua"))
print(cyrtranslit.to_latin("Жадан і Собаки", "ua"))

Згадан і Собаки
Žadan i Sobaky


## Title normalization

In [ ]:
dup_sample = yt_df[yt_df["audio_path"] == "audio\Крихітка Цахес - На Першому Місці.mp3"]
dup_sample

,title,artist,audio_path,yt_title,yt_url,yt_duration,yt_views
750,На першому місці,Крихітка Цахес,audio\Крихітка Цахес - На Першому Місці.mp3,Крихітка Цахес - На Першому Місці,https://youtube.com/watch?v=RT6BXmAArYc,212.0,102050.0
771,Ти на першому місці,Крихітка Цахес,audio\Крихітка Цахес - На Першому Місці.mp3,Крихітка Цахес - На Першому Місці,https://youtube.com/watch?v=RT6BXmAArYc,212.0,102050.0
777,На першому місці,Крихітка Цахес,audio\Крихітка Цахес - На Першому Місці.mp3,Крихітка Цахес - На Першому Місці,https://youtube.com/watch?v=RT6BXmAArYc,212.0,102050.0
782,На першому мiсцi,Крихітка Цахес,audio\Крихітка Цахес - На Першому Місці.mp3,Крихітка Цахес - На Першому Місці,https://youtube.com/watch?v=RT6BXmAArYc,212.0,102050.0
800,Ty na pershomu misci,Крихітка Цахес,audio\Крихітка Цахес - На Першому Місці.mp3,Крихітка Цахес - На Першому Місці,https://youtube.com/watch?v=RT6BXmAArYc,212.0,102051.0
801,На першому місті,Крихітка Цахес,audio\Крихітка Цахес - На Першому Місці.mp3,Крихітка Цахес - На Першому Місці,https://youtube.com/watch?v=RT6BXmAArYc,212.0,102051.0
814,На першому місті,Крихітка Цахес,audio\Крихітка Цахес - На Першому Місці.mp3,Крихітка Цахес - На Першому Місці,https://youtube.com/watch?v=RT6BXmAArYc,212.0,102051.0


In [18]:
dup_sample["title"].value_counts()

На першому місці        1
Ти на першому місці     1
На першому  місці       1
На першому мiсцi        1
Ty na pershomu misci    1
На першому місті        1
На першому  місті       1
Name: title, dtype: int64

### Latin to cyrillic
Handle the case with mix latin and cyrillic letters and convert all to cyrillic ([GitHub](https://github.com/opendatakosovo/cyrillic-transliteration)):

In [7]:
import cyrtranslit
import unicodedata

def fix_mixed_cyrillic(text):
    text = unicodedata.normalize("NFKC", text)
    return cyrtranslit.to_cyrillic(text, "ua")

assert "Воплі Відоплясова" == fix_mixed_cyrillic("Воплi Відоплясoва")

In [ ]:
yt_df['title_norm'] = yt_df['title'].apply(fix_mixed_cyrillic)

yt_df[yt_df["audio_path"] == "audio\Крихітка Цахес - На Першому Місці.mp3"]["title_norm"].value_counts()

На першому місці        2
Ти на першому місці     1
На першому  місці       1
Ти на персгому місці    1
На першому місті        1
На першому  місті       1
Name: title_norm, dtype: int64

### Brackets

In [38]:
import re

def remove_brackets(string: str):
    return re.sub(r'\[.*?\]|\(.*?\)|\{.*?\}', '', string)

some_str = "Hello [Official version]"
assert remove_brackets(some_str) == "Hello "

### White spaces
Remove trailing white spaces:

In [39]:
def normalize_whitespace(text):
    return re.sub(r"\s+", " ", text).strip()

assert "a b c" == normalize_whitespace(" a    b   c ")

In [52]:
yt_df['title_norm'] = yt_df['title_norm'].apply(normalize_whitespace)

yt_df[yt_df["audio_path"] == "audio\Крихітка Цахес - На Першому Місці.mp3"]["title_norm"].value_counts()

На першому місці        3
На першому місті        2
Ти на першому місці     1
Ти на персгому місці    1
Name: title_norm, dtype: int64

### String similarity
Consider the level of string similarity. We choose partial_ratio as it considers substrings and calculates their similarity, which is more suitable in our case:

In [58]:
from rapidfuzz import fuzz

titles = [
    "На першому місці",
    "На першому місті",
    "Ти на першому місці",
    "Ти на персгому місці"
]

print("WRatio:")   
for i in range(len(titles)):
    for j in range(i + 1, len(titles)):
        print(f"Similarity({titles[i]} <-> {titles[j]}) = {fuzz.WRatio(titles[i], titles[j]):.2f}")

print("\nPartial ratio:")
for i in range(len(titles)):
    for j in range(i + 1, len(titles)):
        print(f"Similarity({titles[i]} <-> {titles[j]}) = {fuzz.partial_ratio(titles[i], titles[j]):.2f}")

WRatio:
Similarity(На першому місці <-> На першому місті) = 93.75
Similarity(На першому місці <-> Ти на першому місці) = 85.71
Similarity(На першому місці <-> Ти на персгому місці) = 77.78
Similarity(На першому місті <-> Ти на першому місці) = 80.00
Similarity(На першому місті <-> Ти на персгому місці) = 72.22
Similarity(Ти на першому місці <-> Ти на персгому місці) = 92.31

Partial ratio:
Similarity(На першому місці <-> На першому місті) = 93.75
Similarity(На першому місці <-> Ти на першому місці) = 96.77
Similarity(На першому місці <-> Ти на персгому місці) = 87.50
Similarity(На першому місті <-> Ти на першому місці) = 90.32
Similarity(На першому місті <-> Ти на персгому місці) = 81.25
Similarity(Ти на першому місці <-> Ти на персгому місці) = 89.47


Checking the correspondence of the found YT title to the song title seems to be working pretty good (TODO: then the rest of the songs with high similarity score need to be merged into one):

In [74]:
song_titles = yt_df[yt_df["audio_path"] == 'audio\\Плач Єремії - Ти втретє цього літа зацвітеш.mp3']["title_norm"].values
yt_title = yt_df[yt_df["audio_path"] == 'audio\\Плач Єремії - Ти втретє цього літа зацвітеш.mp3']["yt_title"].values[0]

for title in song_titles:
    print(f"Similarity({title} <-> {yt_title}) = {fuzz.WRatio(title, yt_title):.2f}")

Similarity(Ти втретє цього літа зацвітеш <-> Плач Єремії - Ти втретє цього літа зацвітеш) = 95.00
Similarity(Літо <-> Плач Єремії - Ти втретє цього літа зацвітеш) = 30.00
Similarity(Ти втретє цього літа зацвітеш <-> Плач Єремії - Ти втретє цього літа зацвітеш) = 95.00
Similarity(Ти втретє цього літа зацвітеш <-> Плач Єремії - Ти втретє цього літа зацвітеш) = 95.00
Similarity(Ті, що забули <-> Плач Єремії - Ти втретє цього літа зацвітеш) = 32.88
Similarity(Ти втретє цього літа зацвітеш <-> Плач Єремії - Ти втретє цього літа зацвітеш) = 95.00


Some cornercases: here the "Твої ало, мої але" contains both words "Ало-але" and is probably still the same song despite the low similarity:

In [70]:
song_titles = yt_df[yt_df["audio_path"] == 'audio\\Ало-але.mp3']["title_norm"].values
yt_title = yt_df[yt_df["audio_path"] == 'audio\\Ало-але.mp3']["yt_title"].values[0]

for title in song_titles:
    print(f"Similarity({title} <-> {yt_title}) = {fuzz.WRatio(title, yt_title):.2f}")

Similarity(Ало-але <-> Ало-але) = 100.00
Similarity(Ало але <-> Ало-але) = 85.71
Similarity(Алло, але <-> Ало-але) = 75.00
Similarity(Твої ало, мої але <-> Ало-але) = 55.38
Similarity(Ало, але <-> Ало-але) = 80.00


In [10]:
from rapidfuzz import fuzz

def similarity_score(str_1: str, str_2: str):
    str_1, str_2 = str(str_1).lower(), str(str_2).lower()
    similarity_score = fuzz.WRatio(str_1, str_2)
    return similarity_score

similarity_score("На першому місці", "Ти на першому місті")

85.71428571428572

### Similarity score on our dataset

In [ ]:
yt_df["title-yt_title score"] = yt_df.apply(lambda row: similarity_score(row["title"], row["yt_title"]), axis=1)
yt_df["title-yt_title score"].describe()

count    4341.000000
mean       81.891261
std        24.347289
min         0.000000
25%        85.500000
50%        90.000000
75%        96.000000
max       100.000000
Name: title-yt_title score, dtype: float64

In [93]:
yt_df[(yt_df["title-yt_title score"] < 70) & (yt_df["title-yt_title score"] > 60)][["artist", "title", "yt_title", "title-yt_title score", "yt_url"]]

,artist,title,yt_title,title-yt_title score,yt_url
368,Один В Каное,Піши,Один в каное - Пішы,67.500000,https://youtube.com/watch?v=6L5-Psa4K_o
381,Один В Каное,дощi,Один в каное - Дощі,67.500000,https://youtube.com/watch?v=JBfCO_t9pmk
458,Друга Ріка,День і ніч,Друга Ріка - Денніч,67.500000,https://youtube.com/watch?v=eyacuWfV88c
629,Воплі Відоплясова,Свiт,Воплі Відоплясова - Світ,67.500000,https://youtube.com/watch?v=t9Z7LR5uBS0
639,Воплі Відоплясова,Що було,Воплі Відоплясова - Були на селі,61.071429,https://youtube.com/watch?v=Kk6uv80Slj8
1305,Димна Суміш,Допобачення любов,"До побачення, кохання",68.421053,https://youtube.com/watch?v=wT0rh7_XhBc
1331,Воплi Вiдоплясова,Свiт,Воплі Відоплясова - Світ,67.500000,https://youtube.com/watch?v=t9Z7LR5uBS0
1336,Воплi Вiдоплясова,Що було,Воплі Відоплясова - Були на селі,61.071429,https://youtube.com/watch?v=Kk6uv80Slj8
1452,Klavdia Petrivna,Пам‘ятай,Klavdia Petrivna — Пам'ятай (техно),60.000000,https://youtube.com/watch?v=iSnYw0jmwek
1542,Khrystyna Soloviy,Коала - Instrumental,NaN,60.000000,NaN


In [94]:
yt_df[(yt_df["title-yt_title score"] < 60) & (yt_df["title-yt_title score"] > 50)][["artist", "title", "yt_title", "title-yt_title score", "yt_url"]]

,artist,title,yt_title,title-yt_title score,yt_url
375,Один В Каное,Шахи - Наживо,Один в каное - Шахи,60.000000,https://youtube.com/watch?v=AMgRGZehGeU
653,Воплі Відоплясова,Солнечные дни,Воплі Відоплясова - Сонячні дні [Official Video],57.000000,https://youtube.com/watch?v=Pm0h5on95Kw
745,Мертвий Півень,Beautifull карпати,Б'ютіфул Карпати,57.826087,https://youtube.com/watch?v=u-ymCwdgSIY
813,Крихітка Цахес,"Твої ало, мої але",Ало-але,55.384615,https://youtube.com/watch?v=xH6cxIXtGWI
1028,Artem Pivovarov,"О, Панно!","Артем Пивоваров & The Вуса - О, Панно (feat. Д...",57.000000,https://youtube.com/watch?v=7egk1oND8x8
1080,Artem Pivovarov,Дежавю - Acoustic Version,Артем Пивоваров - Дежавю (UA Version),55.161290,https://youtube.com/watch?v=D83kYtgLZpI
1222,Кому Вниз,Лірник,Кому Вниз - Ліра (Украина),51.300000,https://youtube.com/watch?v=7hcatw4aTGo
1321,Воплi Вiдоплясова,Солнечные дни,Воплі Відоплясова - Сонячні дні [Official Video],57.000000,https://youtube.com/watch?v=Pm0h5on95Kw
1450,Klavdia Petrivna,Уже світає а мене немає,Klavdia Petrivna — Уже світає,57.575758,https://youtube.com/watch?v=9kdk-xdKsE8
1718,Плач Єремії,А вона лиш вона,◾Плач Єремії ◾ Вона ◾,54.285714,https://youtube.com/watch?v=cKIX5yZEm2g


In [ ]:
yt_df[(yt_df["title-yt_title score"] < 50) & (yt_df["title-yt_title score"] > 40)][["artist", "title", "yt_title", "title-yt_title score", "yt_url"]]

,artist,title,yt_title,title-yt_title score,yt_url
78,Go_A,Tenera,Go-A - ЖАЛЬМЕНІНА [ZHALMENINA] (Audio),45.000000,https://youtube.com/watch?v=4vEeYQ9wwuc
80,Go_A,GoaShum,Go_A - SHUM - Ukraine 🇺🇦 Official Music Video ...,40.714286,https://youtube.com/watch?v=U7-dxzp6Jvs
300,Океан Ельзи,Кiшка,Океан Ельзи - Кішка | Kishka (official video),48.000000,https://youtube.com/watch?v=QQ_GAf_g72I
366,Один В Каное,Пиши,Один в каное - Пішы,45.000000,https://youtube.com/watch?v=6L5-Psa4K_o
382,Один В Каное,пiши,Один в каное - Пішы,45.000000,https://youtube.com/watch?v=6L5-Psa4K_o
480,Vivienne Mort,Іній,"Vivienne Mort — Іній [single, 2017]",45.000000,https://youtube.com/watch?v=8S9x47MLq0Q
549,Vivienne Mort,Veсhir,Vivienne Mort — Вечір,45.000000,https://youtube.com/watch?v=0uQny5u6q8Q
1046,Artem Pivovarov,Мираж,АРТЕМ ПИВОВАРОВ - МІРАЖ [UA Version] (Прем'єра...,48.000000,https://youtube.com/watch?v=ZYYDVQNmrBs
1139,СТРУКТУРА ЩАСТЯ,🄴 Сидативи,SadSvit - Силуети (feat. СТРУКТУРА ЩАСТЯ) Lyri...,42.750000,https://youtube.com/watch?v=NjDMzkRbSZI
1194,Кому Вниз,Попiл,"#02. КОМУ ВНИЗ - Попіл. ""Ретроспектива"" у Льві...",48.000000,https://youtube.com/watch?v=IElR7110ug0


In [95]:
yt_df[yt_df["title-yt_title score"] < 50][["artist", "title", "yt_title", "title-yt_title score"]]

,artist,title,yt_title,title-yt_title score
78,Go_A,Tenera,Go-A - ЖАЛЬМЕНІНА [ZHALMENINA] (Audio),45.000000
80,Go_A,GoaShum,Go_A - SHUM - Ukraine 🇺🇦 Official Music Video ...,40.714286
103,Бумбокс,TNT,ТНТ,0.000000
180,alyona alyona,Мамо,alyona alyona & Jerry Heil - Teresa & Maria (L...,0.000000
198,alyona alyona,Zozulia,alyona alyona & Jerry Heil & Ginger Mane - Зозуля,27.692308
...,...,...,...,...
4239,TVORCHI,You,TVORCHI - Heart Of Steel (Eurovision Version) ...,20.000000
4304,Антитіла,Амореморе,АНТИТІЛА - НЕВИДИМКА (Official HD Ukrainian vers),28.500000
4326,Антитіла,Мила,Ed Sheeran – 2step ft Antytila [Official Video],0.000000
4337,Антитіла,Балерина,Ed Sheeran – 2step ft Antytila [Official Video],0.000000


# Final normalization and filtering

Final approach based on the findings from previous steps:

1. **Non-unique audio_path**: get rid of duplicates of the same song (leave only one song)
2. **Unrelevant audio_path**: mark songs whose title doesn't match the fetched YT audio title

## Dataset merging and preprocessing

First let's prepare the dataframe with metadata about the songs and merge it with the YT dataframe:

In [161]:
songs_df = pd.read_csv(os.path.join("data", "songs_filtered_v1.csv"))
yt_df = pd.read_csv(os.path.join("data", "yt_songs_filtered_v1.csv"))
merged_df = pd.merge(songs_df, yt_df, on=["title", "artist"], how="inner")

assert len(merged_df) == len(yt_df) == len(songs_df)
print("Number of samples before filtering: ", len(merged_df))
print("Columns: ", merged_df.columns.values)
print("Number of uniques audio_path:", len(merged_df["audio_path"].unique()))

Number of samples before filtering:  4341
Columns:  ['title' 'artist' 'title_listeners' 'title_scrobbles' 'title_tags'
 'title_duration' 'audio_path' 'yt_title' 'yt_url' 'yt_duration'
 'yt_views']
Number of uniques audio_path: 3227


In [162]:
merged_df[merged_df["artist"] =="Mad Heads XL"]["title_tags"].value_counts()

['ska', 'ukrainian', 'reggae', 'rock', 'ethnic', 'ska', 'ukrainian', 'reggae']    69
['ska', 'ska', 'ukrainian', 'reggae']                                              2
['ukrainian', 'reggae', 'ska', 'ukrainian', 'reggae']                              1
['ukrainian', 'positive', 'ska', 'ukrainian', 'reggae']                            1
['pop-rock', 'ska', 'ukrainian', 'reggae']                                         1
['ska', 'ska punk', 'ska', 'ukrainian', 'reggae']                                  1
['covers', 'positive', 'ska', 'ukrainian', 'reggae']                               1
['ukrainian', 'folk rock', 'folk song', 'ska', 'ukrainian', 'reggae']              1
['ukrainian rock', 'ska', 'ukrainian', 'reggae']                                   1
['feel good', 'ska', 'ukrainian', 'reggae']                                        1
['ska-punk', 'ska', 'ukrainian', 'reggae']                                         1
['ukrainian', 'punk rock', 'ska', 'ukrainian', 'reggae']         

Normalize the arist names and song titles:

In [163]:
import unicodedata

merged_df = normalize_artist(merged_df)
merged_df["title"] = merged_df["title"].apply(lambda x: normalize_whitespace(unicodedata.normalize("NFKC", x)))

Convert popularity and duration data to integers:

In [3]:
def str_to_int(s: str):
    if not pd.isnull(s):
        return int(float(str(s).replace(",", "")))

In [ ]:
merged_df["title_listeners"] = merged_df["title_listeners"].apply(str_to_int).astype("Int64")
merged_df["title_scrobbles"] = merged_df["title_scrobbles"].apply(str_to_int).astype("Int64")
merged_df["yt_views"] = merged_df["yt_views"].apply(str_to_int).astype("Int64")
merged_df["yt_duration"] = merged_df["yt_duration"].apply(str_to_int).astype("Int64")

In [2]:
def duration_to_int(duration: str):
    if pd.isnull(duration):
        return None
    if isinstance(duration, int) or isinstance(duration, float):
        return int(duration)
    parts = duration.split(":")
    if parts[0].isdigit() and parts[1].isdigit():
        return int(parts[0]) * 60 + int(parts[1])

In [ ]:
merged_df["title_duration"] = merged_df["title_duration"].apply(duration_to_int).astype("Int64")
merged_df["yt_duration"] = merged_df["yt_duration"].apply(duration_to_int).astype("Int64")

### Title genre tags

Preprocess tags: filter only relevant once and prepare column with "most common" genre tags for each song:

In [166]:
merged_df["title_tags"] = merged_df["title_tags"].apply(eval)

In [167]:
all_tags = set()
for tags in merged_df["title_tags"].values:
    all_tags.update(tags)

print("Total number of unique tags: ", len(all_tags))

Total number of unique tags:  437


In [168]:
# Save all tags to the file
with open('configs/tags_v1.txt', 'w') as f:
    all_tags = sorted(list(all_tags))
    for tag in all_tags:
        f.write(tag + '\n')

In [169]:
# Read the file with manually filtered genre tags
with open('configs/genre_tags_v1.txt', 'r') as f:
    genre_tags = f.read().split('\n')

print(genre_tags)

['', 'alt. rock', 'alternative', 'alternative metal', 'alternative rock', 'ambient', 'ambient pop', 'art rock', 'ballad', 'blues', 'britpop', 'cabaret', 'chanson', 'christmas', 'classic', 'classic rock', 'classical', 'comedy', 'country', 'dark electro', 'depressive rock', 'doom metal', 'ebm', 'edm', 'electric buzz', 'electrofolk', 'electronic', 'electronic rock', 'electronica', 'electropop', 'emotional rock', 'ethereal', 'ethnic', 'ethno', 'ethno-ska', 'experimental', 'experimental rock', 'finnish punk', 'finskii religioznij rock', 'folk', 'folk metal', 'folk pop', 'folk rock', 'folk song', 'folk-punk', 'folk-rock', 'folk-ska', 'folk/rock', 'folktronica', 'funk', 'funky-groove', 'gop rock', 'goth rock', 'gothic', 'gothic love metal', 'gothic metal', 'gothic rock', 'groove', 'grunge', 'gypsy punk', 'hard rock', 'hardcore punk', 'hip hop', 'hip-hop', 'hyperpop', 'indie', 'indie folk', 'indie pop', 'indie rock', 'indie-rock', 'indiepop', 'industrial metal', 'instrumental', 'italo disco', 

In [170]:
set(all_tags) - set(genre_tags)

{'#1',
 '00s',
 '1',
 '10 of 10 stars',
 '10s',
 '1994',
 '2 stars',
 '2000',
 '2004',
 '2005',
 '2007',
 '2010s',
 '2013',
 '2014',
 '2015',
 '2016',
 '2018',
 '2019',
 '2021',
 '2rika',
 '3',
 '3 stars',
 '4 stars',
 '4-5',
 '4dnb',
 '5 stars',
 '80s',
 '90s',
 'a cappella',
 'about me',
 'acapella',
 'acoustic',
 'acoustic guitar',
 'addictive',
 'all',
 'alone with the woods',
 'altair',
 'autumn2011',
 'awesome song',
 'ballade',
 'ballads',
 'bbpe',
 'beautiful',
 'besause i lovvvvve it',
 'booba',
 'boombox',
 'boring',
 'breakbeat',
 'buttons',
 'calm',
 'catherine and coskun',
 'chart may 2013',
 'chill out',
 'chill-out tunes',
 'chillout',
 'christmas 2019',
 'christmas songs',
 'cities',
 'cloudwalkers',
 'cool',
 'cosmos',
 'cover',
 'covers',
 'credo',
 'crossover',
 'dance',
 'dark',
 'dark psytrance',
 'darkwave',
 'dead rooster',
 'depressive',
 'digital',
 'dnb',
 'doom rock',
 'drive',
 'druga rika',
 'drum and bass',
 'drum n bass',
 'dwell',
 'eastbeat',
 'easy lis

In [171]:
def filter_tags(tags):
    return sorted(list(set([tag for tag in tags if tag in genre_tags])))

merged_df["title_tags"] = merged_df["title_tags"].apply(filter_tags)

In [172]:
def tag_list_to_common_genres(tag_list):
    common_genres = [
        'pop', 'rock', 'rap', 'jazz', 'folk', 'funk', 'metal', 'punk', 'blues', 'electro',
        'hip-hop', 'indie', 'classical', 'country', 'soul', 'edm', 'lo-fi', 'alternative',
        'reggae', 'groove',  'grunge', 'gothic', 'ska', 'disco'
    ]
    with open('configs/common_genre_tags_v1.txt', 'w') as f:
        for genre in common_genres:
            f.write(genre + '\n')
    common_tags = []
    for tag in tag_list:
        tag = tag.replace('hip hop', 'hip-hop')
        for genre in common_genres:
            if genre in tag:
                common_tags.append(genre)
    return sorted(list(set(common_tags)))

merged_df['genre_tags'] = merged_df['title_tags'].apply(tag_list_to_common_genres)
merged_df['genre_tags'].head(10)

0      [alternative, pop, rock]
1    [alternative, blues, rock]
2                        [rock]
3                        [rock]
4                 [indie, rock]
5            [indie, pop, rock]
6                        [rock]
7           [alternative, rock]
8                        [rock]
9                        [rock]
Name: genre_tags, dtype: object

## Filtering duplicates after title normalization

Normalize song titles and fetched yt audio titles: cyrillic --> (?) brackets --> white spaces:

In [173]:
def normalize_title(title: str):
    if pd.isnull(title):
        return title
    title = str(title).lower()
    return normalize_whitespace(fix_mixed_cyrillic(title))

merged_df["title_norm"] = merged_df["title"].apply(normalize_title)
merged_df["yt_title_norm"] = merged_df["yt_title"].apply(normalize_title)

Filter duplicates of the same song for each artist separately:

In [174]:
filtered_df = merged_df.dropna(subset=["title", "artist", "audio_path", "yt_url"])

filtered_df = (
    filtered_df.sort_values(by="title_scrobbles", ascending=False)
        .drop_duplicates(subset=["artist", "title_norm"], keep="first")
        .sort_index()  # Restore original order
)

print("Number of samples after filtering: ", len(filtered_df))

Number of samples after filtering:  3960


## Similarity score filtering

Now let's consider the similarity score to check the relevance of the fetched audio:

1) Calculate the similarity score (title, yt_title)
2) Mark songs with similarity score below the threshold as bad_audio (a.k.a the fetched audio doesn't match the song)


In [175]:
TITLE_YT_TITLE_SIMILARITY_THRESHOLD = 60

filtered_df["title-yt_title score"] = filtered_df.apply(lambda row: similarity_score(row["title_norm"], row["yt_title_norm"]), axis=1)
filtered_df["bad_audio"] = filtered_df["title-yt_title score"] < TITLE_YT_TITLE_SIMILARITY_THRESHOLD

3) Group dataset by audio_path (and artist) and consider those with count > 1 (a.k.a. possibly have duplicates)
4) Sort songs in the popularity order
5) Consider only songs with high similarity: mark all as duplicates except for one song with the highest popularity

In [176]:
audio_path_groups = filtered_df.groupby("audio_path").filter(lambda x: len(x) > 1)
print("Number of duplicated audio paths after filtering: ", len(audio_path_groups))

Number of duplicated audio paths after filtering:  1286


In [177]:
filtered_df["duplicate"] = False

for audio_path, group in audio_path_groups.groupby("audio_path"):
    for artist, artist_group in group.groupby("artist"):
        if len(artist_group) == 1:
            continue
        artist_group = artist_group.sort_values(by="title_scrobbles", ascending=False)
        most_popular_relevant_idx = False
        for idx, row in artist_group.iterrows():
            if row["title-yt_title score"] >= TITLE_YT_TITLE_SIMILARITY_THRESHOLD:
                if not most_popular_relevant_idx:
                    most_popular_relevant_idx = idx  # Keep the one with the highest title_scrobbles
                else:
                    filtered_df.loc[idx, "duplicate"] = True

In [178]:
print("Number of duplicates:", filtered_df["duplicate"].sum())
print("Number of bad audio:", filtered_df["bad_audio"].sum())

Number of duplicates: 499
Number of bad audio: 295


In [179]:
relevant_df = filtered_df[~filtered_df["duplicate"] & ~filtered_df["bad_audio"]]
print("Number of relevant samples: ", len(relevant_df))

Number of relevant samples:  3166


## Save merged dataset and update summary

In [180]:
columns_to_save = [
    "title", "artist", "title_listeners", "title_scrobbles", "title_tags", "genre_tags",
    "title_duration", "audio_path", "yt_title", "yt_url", "yt_duration", "yt_views"
]

relevant_df[columns_to_save].to_csv(os.path.join("data", "merged_songs_v1.csv"), index=False)

In [182]:
print("Number of ukr artists: ", len(relevant_df["artist"].unique()))

Number of ukr artists:  60


In [181]:
songs_per_artist = relevant_df["artist"].value_counts()

print(f"Total number of songs: {len(relevant_df)}\n")
print(f"Number of songs per artist:\n{songs_per_artist.describe()}")

Total number of songs: 3166

Number of songs per artist:
count    60.000000
mean     52.766667
std      19.916237
min      16.000000
25%      37.000000
50%      53.500000
75%      66.250000
max      94.000000
Name: artist, dtype: float64


## Artists df preprocessing

In [1]:
import pandas as pd
import os

artists_df = pd.read_csv(os.path.join("data", "artists_v1.csv"))
artists_df

,artist,artist_listeners,artist_scrobbles,artist_tags
0,Океан Ельзи,"203,060","13,552,898","['rock', 'ukrainian', 'ukrainian rock', 'indie..."
1,Go_A,"189,946","4,088,960","['ukrainian', 'folktronica', 'folk', 'electron..."
2,Бумбокс,"174,374","6,424,069","['ukrainian', 'funk', 'reggae', 'hip-hop', 'fu..."
3,Скрябін,"59,090","2,821,158","['ukrainian', 'synthpop', 'pop-rock', 'electro..."
4,KALUSH,"106,178","1,850,491","['ukrainian', 'rap', 'hip-hop', 'ukraine', 'eu..."
...,...,...,...,...
201,Dens,"2,485","66,284","['post-hardcore', 'christian', 'melodic hardco..."
202,TELLY GRAVE,"4,942","66,814","['rap', 'hip-hop', 'russian', 'ukraine', 'ukra..."
203,NAZVA,"4,640","60,064","['ukrainian', 'folk', 'folk pop', 'pop', 'ukra..."
204,Dazzle Dreams,"5,948","109,380","['ukrainian', 'electronic', 'electropop', 'amb..."


In [5]:
artists_df["artist_listeners"] = artists_df["artist_listeners"].apply(str_to_int).astype("Int64")
artists_df["artist_scrobbles"] = artists_df["artist_scrobbles"].apply(str_to_int).astype("Int64")

In [6]:
artists_df["artist_tags"] = artists_df["artist_tags"].apply(eval)

In [9]:
unique_artist_tags = set()

for tags in artists_df["artist_tags"].values:
    unique_artist_tags = unique_artist_tags.union(set(tags))

print("Number of unique artists tags:", len(unique_artist_tags))

Number of unique artists tags: 352


In [10]:
unique_artist_tags

{'00s',
 '10s',
 '2',
 '2000s',
 '2005',
 '2009',
 '2010s',
 '2017 single',
 '3',
 '4',
 '6 of 10 stars',
 '90s',
 'a cappella',
 'abstract hip-hop',
 'acapella',
 'acid jazz',
 'acoustic',
 'agri-alco-rock',
 'all',
 'alt rock',
 'alternative',
 'alternative metal',
 'alternative rock',
 'ambient',
 'ambient music',
 'anime',
 'anthem',
 'argentina',
 'art pop',
 'art rock',
 'austria',
 'authentic',
 'avant-folk',
 'avant-garde',
 'babkin',
 'band',
 'bands',
 'baroque pop',
 'based',
 'bayracore',
 'beats',
 'beautiful',
 'bedroom pop',
 'belarus',
 'belarusian',
 'best of preselection',
 'blues',
 'bonzaiprogressive',
 'booba',
 'bosnian',
 'british',
 'britpop',
 'cabaret',
 'celebration',
 'chanson',
 'childrens music',
 'chillhop',
 'chillout',
 'chiptune',
 'christian',
 'christian hardcore',
 'christmas',
 'christmas music',
 'christmas songs',
 'classical',
 'cloud rap',
 'club',
 'codeinoslav',
 'comedy',
 'cover',
 'covers',
 'cringecore',
 'cute',
 'cyber-folk',
 'dance',


In [11]:
artists_df.to_csv(os.path.join("data", "artists_v1.csv"), index=False)

## Album df preprocessing

In [1]:
import pandas as pd
import os

album_df = pd.read_csv(os.path.join("data", "albums_v1.csv"))

print("Number of albums:", len(album_df))
album_df.columns

Number of albums: 1086


Index(['artist', 'album', 'num_tracks', 'album_length', 'release_date',
       'listeners', 'scrobbles', 'tags', 'tracks'],
      dtype='object')

### 1. Filter albums by the number of tracks

In [6]:
print("No info about number of tracks:", album_df["num_tracks"].isnull().sum())
print("Number of tracks with at least 2 songs:", len(album_df[album_df["num_tracks"] > 1]))
print("Number of tracks with exactly 2 songs:", len(album_df[album_df["num_tracks"] == 2]))
print("Number of tracks with more than 20 songs:", len(album_df[album_df["num_tracks"] > 20]))

No info about number of tracks: 139
Number of tracks with at least 2 songs: 449
Number of tracks with exactly 2 songs: 27
Number of tracks with more than 20 songs: 27


In [8]:
album_df[album_df["num_tracks"] == 2]

,artist,album,num_tracks,album_length,release_date,listeners,scrobbles,tags,tracks
24,Go_A,Solovey - Single,2.0,5:52,4 March 2020,640,"8,161","['ukrainian', 'folktronica', 'folk', 'electron...","['Solovey', 'Solovey (Instrumental Version)']"
25,Go_A,Solovey,2.0,5:52,16 December 2021,"39,867","403,092","['eurovision', 'eurovision song contest', 'eur...","['Solovey', 'Solovey (Instrumental Version)']"
33,Go_A,Рано-раненько,2.0,6:43,2019,"16,785","118,434","['folktronica', 'ukrainian', 'folk', 'electron...","['Рано-раненько', 'Рано-раненько (Радіо Версія)']"
40,Go_A,Dumala,2.0,6:05,"2 tracks, 6:05","5,260","50,657","['ukrainian', 'folktronica', 'folk', 'electron...","['Dumala', 'Dumala (Speed Up version)']"
41,Go_A,Dumala - Single,2.0,6:05,23 November 2023,57,690,"['ukrainian', 'folktronica', 'folk', 'electron...","['Dumala', 'Dumala (Speed Up version)']"
42,Go_A,Krip,2.0,7:04,"2 tracks, 7:04","4,617","29,606","['ukrainian', 'folktronica', 'folk', 'electron...","['Krip', 'Krip (Instrumental)']"
43,Go_A,Krip - Single,2.0,7:04,24 April 2024,47,335,"['ukrainian', 'folktronica', 'folk', 'electron...","['Krip', 'Krip (Instrumental)']"
71,The Hardkiss,Жива,2.0,8:10,25 September 2019,"2,779","28,709","['ukrainian', 'electronic', 'dance', 'alternat...","['Жива', 'Жива (Instrumental)']"
83,The Hardkiss,Free Me - Single,2.0,6:24,3 August 2018,138,591,"['ukrainian', 'electronic', 'dance', 'alternat...","['Free Me', 'Free Me - Instrumental Version']"
85,The Hardkiss,"Хто, як не ти",2.0,7:06,13 August 2019,690,"2,862","['ukrainian', 'electronic', 'dance', 'alternat...","['Хто, як не ти', 'Хто, як не ти (Instrumental..."


In [6]:
album_df[album_df["num_tracks"] > 20]

,artist,album,num_tracks,album_length,release_date,listeners,scrobbles,tags,tracks
0,Океан Ельзи,1221,21.0,NaN,7 December 2006,"29,753","426,580","['rock', 'alternative', 'rock', 'ukrainian', '...","['Long Time Ago', 'Вона підійшла до вікна', 'Т..."
5,Океан Ельзи,The Best Of,21.0,78:50,6 August 2010,"15,088","231,409","['rock', 'ukrainian', 'ukrainian rock', 'indie...","['Я так хочу...', 'Все буде добре', 'Без бою',..."
14,Океан Ельзи,Самое лучшее,22.0,82:04,25 May 2002,930,"4,410","['rock', 'ukrainian', 'ukrainian rock', 'indie...","['Там, де нас нема', '911', 'Я до тебе', 'Ти с..."
15,Океан Ельзи,Избранное,40.0,159:09,26 September 2013,"1,746","38,580","['rock', 'ukrainian', 'ukrainian rock', 'indie...","['Там, де нас нема', '911', 'Без бою', 'Сосни'..."
16,Океан Ельзи,Найкраще,34.0,135:05,2014,886,"40,058","['rock', 'ukrainian', 'ukrainian rock', 'indie...","['Там де нас нема', 'Ластiвка З Твого Мiста', ..."
17,Океан Ельзи,найкраще,34.0,135:05,2014,886,"40,058","['rock', 'ukrainian', 'ukrainian rock', 'indie...","['Там де нас нема', 'Ластiвка З Твого Мiста', ..."
49,Бумбокс,Краще,55.0,NaN,2010,"3,109","138,848","['ukrainian', 'funk', 'reggae', 'hip-hop', 'sa...","['Бета-каротин', 'День', 'Діагноз', 'E-Mail', ..."
111,Один В Каное,Один в каное,25.0,NaN,23 January 2016,"15,697","519,215","['ukrainian', 'ukraine', 'odin v kanoe', '3', ...","['Небо', 'Янголе', 'Дерево', 'Бути тобою', 'Ша..."
142,Друга Ріка,Best,21.0,NaN,14 April 2019,486,"7,772","['ukrainian', 'rock', 'ukrainian rock', 'pop-r...","['Догоним! Доженемо!', 'Деннiч', 'Так мало тут..."
220,Брати Гадюкіни,Вродило! (Live 2006),21.0,96:57,2006,24,401,"['ukrainian', 'folk', 'country', 'rock', 'ukra...","['Інтро', 'Було не любити', 'арівідерчі, рома'..."


We can see some good album examples with number of songs higher than 20 ("1221", all albums from Марія Бурмак), while albums with 2 songs usually contain two versions of one song. Thus we will filter only albums with more than 3 songs:

In [ ]:
filtered_df = album_df[album_df["num_tracks"] > 2]

### 2. Filter albums with unique track lists:

We would like to filter albums that have non-unique track list (a.k.a, are the duplicates of the same album).

In [10]:
print("Target album num:", len(filtered_df["tracks"].apply(str).unique()))

Target album num: 378


Let's create a mapping of unique tracks to the first album they appear in:

In [ ]:
unique_tracks_mapping = {}
album_name_mapping = {}
for idx, row in filtered_df.iterrows():
    tracks_str = str(row["tracks"])
    if tracks_str not in unique_tracks_mapping:
        unique_tracks_mapping[tracks_str] = row["album"]
        album_name_mapping[row["album"]] = row["album"]
    else:
        album_name_mapping[row["album"]] = unique_tracks_mapping[tracks_str]


print("Number of unique track lists:", len(unique_tracks_mapping))
print("Number of albums:", len(album_name_mapping))
print("Number of unique albums:", len(set(album_name_mapping.values())))

Number of unique track lists: 378
Number of albums: 414
Number of unique albums: 370


In [10]:
# Filter rows to keep only the first occurrence of unique tracks
filtered_df = filtered_df.drop_duplicates(subset=["tracks"], keep="first")

# sort by artist and album, then save
filtered_df = filtered_df.sort_values(by=["artist", "album"])
filtered_df.to_csv(os.path.join("data", "albums_filtered_v1.csv"), index=False)

### 3. Manual albums filtering

The rest of the albums will be filtered manually. To make the filtering process recreatable, we prepared the ```album_blacklist.txt``` of the following format:
- {artist},{album} - unrelevant albums, remove from the dataframe;
- {artist},{album_1} -> {album_2} - duplicates of the same album, rename;
- {artist},{album_1} + {album_2} - extended version of the same album, merge those.

In [11]:
with open(os.path.join("configs", "album_blacklist.txt"), "r", encoding="utf-8") as f:
    album_blacklist = f.read().splitlines()

print("Number of albums in blacklist:", len(album_blacklist))
print("Album blacklist:", album_blacklist)

Number of albums in blacklist: 82
Album blacklist: ['BLOOMS CORDA,Fog Lavender [Explicit] -> Fog Lavender', 'BLOOMS CORDA,Monodance bonus tracks + Monodance', 'Go_A,найкращі пісні', 'KALUSH,Для душі [Explicit] -> Для душі - EP', 'Kazka,Untitled', 'Kazka,СВЯТА (The Best Of Kazka)', 'Mad Heads XL,Nadiya Yea -> Надія є', 'Mad Heads XL,Надія Є (Спеціальне Новорічне Видання) -> Надія є', 'O.Torvald,Diller Kaifu II + Diller Kaifu I', 'PROBASS ∆ HARDI,"ДОБРОГО ВЕЧОРА, МИ З УКРАЇНИ! [Explicit]" -> "ДОБРОГО ВЕЧОРА, МИ З УКРАЇНИ!"', 'TVORCHI,13 Waves [Explicit] -> 13 Waves', 'TVORCHI,Heart of Steel - EP -> Heart of Steel', 'TVORCHI,Planet X [Explicit] -> Planet X', 'The Hardkiss,the best', 'The Hardkiss,Zhyva i ne zalizna -> Жива і не залізна', 'The Unsleeping,"Пиріжок з любов’ю, злістю - EP" -> "Пиріжок з любов’ю, злістю"', 'To Eternity,ЕЛЛІ - EP -> ЕЛЛІ', 'Vivienne Mort,Rósa -> Rosa', 'Vivienne Mort,Єсєнтукі LOVE - EP -> Єсєнтукі Love', 'Vivienne Mort,Вибране', 'Vivienne Mort,Театр Pipinó -> Т

In [12]:
blacklist_dict = {}

for entry in album_blacklist:
    entry = entry.strip()
    if " -> " in entry:
        artist_album, new_album = entry.split(" -> ")
    elif " + " in entry:
        artist_album, new_album = entry.split(" + ")
    else:
        artist_album = entry
        new_album = None

    artist, album = artist_album.split(",", 1)
    blacklist_dict[(artist, album)] = new_album

blacklist_dict

{('BLOOMS CORDA', 'Fog Lavender [Explicit]'): 'Fog Lavender',
 ('BLOOMS CORDA', 'Monodance bonus tracks'): 'Monodance',
 ('Go_A', 'найкращі пісні'): None,
 ('KALUSH', 'Для душі [Explicit]'): 'Для душі - EP',
 ('Kazka', 'Untitled'): None,
 ('Kazka', 'СВЯТА (The Best Of Kazka)'): None,
 ('Mad Heads XL', 'Nadiya Yea'): 'Надія є',
 ('Mad Heads XL', 'Надія Є (Спеціальне Новорічне Видання)'): 'Надія є',
 ('O.Torvald', 'Diller Kaifu II'): 'Diller Kaifu I',
 ('PROBASS ∆ HARDI',
  '"ДОБРОГО ВЕЧОРА, МИ З УКРАЇНИ! [Explicit]"'): '"ДОБРОГО ВЕЧОРА, МИ З УКРАЇНИ!"',
 ('TVORCHI', '13 Waves [Explicit]'): '13 Waves',
 ('TVORCHI', 'Heart of Steel - EP'): 'Heart of Steel',
 ('TVORCHI', 'Planet X [Explicit]'): 'Planet X',
 ('The Hardkiss', 'the best'): None,
 ('The Hardkiss', 'Zhyva i ne zalizna'): 'Жива і не залізна',
 ('The Unsleeping',
  '"Пиріжок з любов’ю, злістю - EP"'): '"Пиріжок з любов’ю, злістю"',
 ('To Eternity', 'ЕЛЛІ - EP'): 'ЕЛЛІ',
 ('Vivienne Mort', 'Rósa'): 'Rosa',
 ('Vivienne Mort', 'Єсєн

### 4. Updating songs_df with album data

Now let's update album info in the dataframe with songs:

In [ ]:
songs_df = pd.read_csv(os.path.join("data", "albums_songs_v1.csv")) # album data fetched from lastfm + merged_songs_v1.csv
songs_df["albums"] = songs_df["albums"].apply(eval)
songs_df["albums"].describe()

count     3166
unique     987
top         []
freq      1168
Name: albums, dtype: object

In [15]:
album_df = pd.read_csv(os.path.join("data", "albums_v1.csv")) # data about the fetched albums
album_df["num_tracks"] = album_df["num_tracks"].fillna(0).astype(int)
album_df["num_tracks"].describe()

count    1086.000000
mean        5.151934
std         7.260900
min         0.000000
25%         1.000000
50%         1.000000
75%        10.000000
max        71.000000
Name: num_tracks, dtype: float64

Let's prepare old album - new album mapping for albums with more than 2 tracks that match the album naming filtered in the second step:

In [ ]:
songs_df["albums_filtered"] = None

for i, row in songs_df.iterrows():
    artist = row["artist"]
    new_albums = []
    for album in row["albums"]:
        if album_df[album_df["album"] == album]["num_tracks"].values[0] <= 2:
            continue
        album = album_name_mapping[album] # calculated in the [2.] filtering step
        if (artist, album) not in blacklist_dict:
            new_albums.append(album)
        elif blacklist_dict[(artist, album)]:
            new_albums.append(blacklist_dict[(artist, album)]) # replace with new album name
    songs_df.at[i, "albums_filtered"] = sorted(list(set(new_albums)))

In [16]:
songs_df["albums_filtered"].describe()

count     3166
unique     396
top         []
freq      1570
Name: albums_filtered, dtype: object

As we can see, 1570 songs ended up with empty album column and we got 396 unique album combinations. 

Now let's explore the album combinations distribution for a particular artist and prepare the merging function which will asign one album to each of the songs:

In [17]:
songs_df[songs_df["artist"] == "Mad Heads XL"]["albums_filtered"].value_counts()

[]                      20
[УкраїнSKA]             13
[Forever]               12
[Надія є]                8
[Надія є, УкраїнSKA]     4
Name: albums_filtered, dtype: int64

In [19]:
import pandas as pd

def merge_album_data(df):
    # Explode albums into individual rows for better analysis
    df_expanded = df.explode('albums_filtered')
    
    # Count occurrences of each album
    album_counts = df_expanded['albums_filtered'].value_counts()
    
    # Identify albums that are mentioned alone in the original dataset
    standalone_albums = set(df[df['albums_filtered'].apply(len) == 1]['albums_filtered'].explode().unique())
    
    # Determine which albums have at least 4 songs
    valid_albums = set(album_counts[album_counts >= 4].index)
    
    # Assign each song to an album, prioritizing standalone albums first
    def get_main_album(albums):
        # If an album is standalone, prioritize it
        for album in albums:
            if album in standalone_albums:
                return album
        
        # Keep only albums that meet the minimum threshold
        valid = [album for album in albums if album in valid_albums]
        
        if valid:
            return min(valid, key=lambda album: album_counts.get(album, float('inf')))
        
        return 'Other'
    
    df['main_album'] = df['albums_filtered'].apply(get_main_album)
    
    # Recalculate album counts and reassign albums with <4 songs to "Other"
    # TODO: Modify those to consider artist-album pairs as the album name is not unique
    final_counts = df['main_album'].value_counts()
    df['main_album'] = df['main_album'].apply(lambda album: album if final_counts.get(album, 0) >= 4 else 'Other')
    
    return df

After merging we got 189 unique albums and 1724 songs doesn't have album data:

In [20]:
songs_df_copy = songs_df.copy()
songs_df_copy = merge_album_data(songs_df_copy)
songs_df_copy["main_album"].describe()

count      3166
unique      189
top       Other
freq       1724
Name: main_album, dtype: object

Here is the before-after example of all filtered albums and the final main album column for Океан Ельзи:

In [21]:
songs_df_copy[songs_df_copy["artist"]=="Океан Ельзи"]["albums_filtered"].value_counts()

[Міра]                       12
[Земля]                      11
[Dolce Vita]                 11
[]                           10
[Gloria]                      9
[Суперсиметрія]               5
[Янанебібув]                  5
[Модель]                      4
[Tviй Формат, Янанебібув]     3
[1221, Gloria]                3
[1221, Там, де нас нема]      3
[1221, Янанебібув]            2
[1221, Суперсиметрія]         2
[Без меж]                     2
[Там, де нас нема]            2
[1221, Модель]                1
[1221, Tviй Формат]           1
[Tviй Формат]                 1
Name: albums_filtered, dtype: int64

In [22]:
songs_df_copy[songs_df_copy["artist"]=="Океан Ельзи"]["main_album"].value_counts()

Gloria              12
Міра                12
Other               12
Земля               11
Dolce Vita          11
Янанебібув           7
Суперсиметрія        7
Модель               5
Там, де нас нема     5
Tviй Формат          5
Name: main_album, dtype: int64

Let's print the final distribution for all artists and save the final dataset with albums info:

In [ ]:
for artist in songs_df_copy["artist"].unique():
    print(f"\nArtist: {artist}")
    print(songs_df_copy[songs_df_copy["artist"]==artist]["main_album"].value_counts())


Artist: Океан Ельзи
Gloria              12
Міра                12
Other               12
Земля               11
Dolce Vita          11
Янанебібув           7
Суперсиметрія        7
Модель               5
Там, де нас нема     5
Tviй Формат          5
Name: main_album, dtype: int64

Artist: Go_A
Other    37
Name: main_album, dtype: int64

Artist: Бумбокс
Other            17
Меломанія        10
Середній Вік     10
Family Бiзнес     9
III               8
Термінал Б        8
Всё Включено      5
Люди              4
Name: main_album, dtype: int64

Artist: The Hardkiss
Other                  17
Stones and Honey       15
Залізна ластівка       15
Жива і не залізна       8
Cold Altair             6
Perfection Is a Lie     6
Name: main_album, dtype: int64

Artist: alyona alyona
Other        33
Пушка        11
Galas         9
В хаті МА     6
Name: main_album, dtype: int64

Artist: Один В Каное
Other                      21
Один в каное               15
Live In Museum Of Ideas     8
Name: main_alb

In [ ]:
songs_df_copy[["title", "artist", "albums", "albums_filtered", "main_album"]].to_csv(os.path.join("data", "albums_songs_v1.csv"), index=False)